# 예제 5-10 데이터 구간 분할 

### 라이브러리 불러오기 

In [1]:
import pandas as pd
import numpy as np

### read_csv() 함수로 df 생성 

In [2]:
df = pd.read_csv('./auto-mpg.csv', header=None)

### 열 이름 지정 

In [3]:
df.columns = ['mpg','cylinders','displacement','horsepower','weight',
             'acceleration','model year','origin','name']

### horsepower 열의 누락 데이터 삭제하고 실수형으로 변환 

In [4]:
df['horsepower'].replace('?', np.nan, inplace=True)
df.dropna(subset=['horsepower'], axis=0, inplace=True)
df['horsepower'] = df['horsepower'].astype('float')

### np.histogram 함수로 3개의 bin으로 나누는 경계값의 리스트 

In [5]:
count, bin_dividers = np.histogram(df['horsepower'], bins=3)
bin_dividers

array([ 46.        , 107.33333333, 168.66666667, 230.        ])

나누려는 구간 개수를 bins 옵션에 입력하면 각 구간에 속하는 값의 개수와

경계값 리스트를 반환한다. (46 ~ 107.3, 107.3 ~ 168.6, 168.6 ~ 230 구간)

### 3개의 bin에 이름 지정 

In [6]:
bin_names = ['저출력', '보통출력', '고출력']

### pd.cut 함수로 각 데이터를 3개의 bin에 할당 

In [8]:
df['hp_bin'] = pd.cut(x=df['horsepower'], # 데이터 배열
                     bins=bin_dividers, # 경계값 리스트
                     labels=bin_names, # bin 이름
                     include_lowest=True) # 첫 경계값 포함

### horsepower 열, hp_bin 열의 첫 15행 출력 

In [9]:
df[['horsepower', 'hp_bin']].head(15)

,horsepower,hp_bin
0,130.0,보통출력
1,165.0,보통출력
2,150.0,보통출력
3,150.0,보통출력
4,140.0,보통출력
5,198.0,고출력
6,220.0,고출력
7,215.0,고출력
8,225.0,고출력
9,190.0,고출력


# 예제 5-11 더미 변수 

### hp_bin 열의 범주형 데이터를 더미 변수로 변환 

In [10]:
horsepower_dummies = pd.get_dummies(df['hp_bin'])
horsepower_dummies.head(15)

,저출력,보통출력,고출력
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
5,0,0,1
6,0,0,1
7,0,0,1
8,0,0,1
9,0,0,1


get_dummies()함수를 사용하면, 범주형 변수의 모든 고유값을 새로운 더미 변수로 변환

# 예제 5-12 원핫인코딩 

### sklearn 라이브러리 불러오기 

In [11]:
from sklearn import preprocessing

### 전처리를 위한 encoder 객체 만들기 

In [12]:
label_encoder = preprocessing.LabelEncoder() 
onehot_encoder = preprocessing.OneHotEncoder()

### label encoder로 문자열 범주를 숫자형 범주로 변환 

In [14]:
onehot_labeled = label_encoder.fit_transform(df['hp_bin'].head(15))
onehot_labeled

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 2])

In [15]:
type(onehot_labeled)

numpy.ndarray

### 2차원 행렬로 형태 변경 

In [16]:
onehot_reshaped = onehot_labeled.reshape(len(onehot_labeled), 1)
onehot_reshaped

array([[1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [2]])

In [17]:
type(onehot_reshaped)

numpy.ndarray

### 희소행렬로 변환 

In [19]:
onehot_fitted = onehot_encoder.fit_transform(onehot_reshaped)
onehot_fitted

<15x3 sparse matrix of type '<class 'numpy.float64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [20]:
type(onehot_fitted)

scipy.sparse.csr.csr_matrix

# 예제 5-13 정규화 

In [23]:
df = pd.read_csv('./auto-mpg.csv', header=None)
df.columns = ['mpg','cylinders','displacement','horsepower','weight',
             'acceleration','model year','origin','name']

### horsepower 열의 누락 데이터 삭제 후 실수형 변환 

In [24]:
df['horsepower'].replace('?', np.nan, inplace=True)
df.dropna(subset=['horsepower'], axis=0, inplace=True)
df['horsepower'] = df['horsepower'].astype('float')

### horsepower 열의 통계 요약 정보로 최대값 확인 

In [25]:
df.horsepower.describe()

count    392.000000
mean     104.469388
std       38.491160
min       46.000000
25%       75.000000
50%       93.500000
75%      126.000000
max      230.000000
Name: horsepower, dtype: float64

### horsepower 열의 최대값의 절대값으로 모든 데이터 나누기 

In [27]:
df.horsepower = df.horsepower/abs(df.horsepower.max())
df.horsepower.head()

0    0.565217
1    0.717391
2    0.652174
3    0.652174
4    0.608696
Name: horsepower, dtype: float64

In [29]:
df.horsepower.describe()

count    392.000000
mean       0.454215
std        0.167353
min        0.200000
25%        0.326087
50%        0.406522
75%        0.547826
max        1.000000
Name: horsepower, dtype: float64

### 최대값과 최소값을 뺸 값으로 나누기

In [30]:
min_x = df.horsepower - df.horsepower.min()
min_max = df.horsepower.max() - df.horsepower.min()
df.horsepower = min_x/min_max
df.horsepower.head()

0    0.456522
1    0.646739
2    0.565217
3    0.565217
4    0.510870
Name: horsepower, dtype: float64

In [31]:
df.horsepower.describe()

count    392.000000
mean       0.317768
std        0.209191
min        0.000000
25%        0.157609
50%        0.258152
75%        0.434783
max        1.000000
Name: horsepower, dtype: float64

### 

### 

### 

### 